In [1]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import WebBaseLoader
import bs4


loader = WebBaseLoader(
    web_paths= ("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    )
)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)
splits = text_splitter.split_documents(docs) # text_splitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={"k" : 1, "fetch_k" : 4}
)

bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 2
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever],
                weights=[0.2, 0.8])
ensemble_retriever.invoke("오늘의 증시")

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='이데일리\n\n19분전\n\n\n\n\n\n\n\n\n다시 고개 든 관세 경계감… 코스피·코스닥 하락 출발'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='LGD, 전사 AI 전환 가속…"3년 내 생산성 30% 향상이 목표"\n\nLG디스플레이가 전 사업 영역에 인공지능(AI)을 적용하는 AX((AI Transformation, AI 전환) 추진에 속도를 낸다. 이를 통해 3년 안에 업무 생산성을 30% 향상 시키는 게 목표다. LG디스플레이\n\n\n아이뉴스24\n\n\n\n\n42\n개의 관련뉴스 더보기'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content="NH투자증권은 6일 디어유에 대해 올해 2분기 기대치를 밑도는 실적을 기록했지만 중국 진출로 앞으로 실적이 개선될 것으로 내다봤다. 투자의견 '매수'와 목표주가를 기존 6만5000원으로 유지했다. 디어유의 올해 2분\n\n\n머니투데이\n\n30분전")]

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것입니다.
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다.
각 질문은 새 줄로 구분하여 제공하세요. 원본 질문: {question}
"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model_name="chatgpt-4o-latest", temperature=0)
    | StrOutputParser()
    | (lambda x : x.split("\n"))
)

from langchain.load import dumps, loads
def reciprocal_rank_fusion(results, k=60, top_n=2):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(docs)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)
   
    reranked_results = [ (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x : x[1], reverse=True)
    ]
    return reranked_results[:top_n]

chain = generate_queries | ensemble_retriever.map() | reciprocal_rank_fusion
docs = chain.invoke("오늘의 증시")

/tmp/ipykernel_7880/21748928.py:30: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  reranked_results = [ (loads(doc), score)


In [6]:
docs

[([Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='코스피 지수가 다시 3200선 아래로 내려앉았다. 전장에서 기술적 반등에 성공했지만 세제개편안 후폭풍이 여전한 데다 미국의 관세 영향이 가시화되며 뉴욕증시 3대 지수가 일제히 하락한 영향으로 해석된다. 6일 엠피닥터\n\n\n이데일리\n\n12분전'),
   Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='쿠팡의 올해 2분기 신선식품 부문 매출이 전년 동기보다 25% 증가한 것으로 나타났다. 특히 농산물·육류·해산물 등 주요 품목군 확대가 매출 증가에 영향을 준 것으로 보인다. 김범석 쿠팡Inc 의장은 6일(한국시각)\n\n\n뉴시스\n\n\n\n\n60\n개의 관련뉴스 더보기'),
   Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='서울신문\n\n10분전\n\n\n\n\n\n\n\n\n[서울데이터랩]개장 직후 인기 검색 종목 20選')],
  0.049189141547682),
 ([Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='지난해 퇴직연금 실물이전 제도가 시작된 이후 가장 많은 퇴직연금이 몰린 증권사는 한국투자증권인 것으로 확인됐다. 퇴직연금 실물이전은 계좌 내 운용 중인 상품을 해지하지 않고 다른 금융사로 이전할 수 있도록 한 제도다\n\n\n머니투데이\n\n27분전'),
   Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='LGD, 전사 AI 전환 가속…"3년 내 생산성 30% 향상이 목표"\n\nLG디스플레이

In [7]:
from langchain_core.runnables import RunnablePassthrough
template = """다음 맥락을 바탕으로 질문에 답변할 것
{context}
질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
final_chain = (
    {
        "context" : chain,
        "question" : RunnablePassthrough()
    }
    | prompt | ChatOpenAI(model_name="chatgpt-4o-latest", temperature=0)
    | StrOutputParser()
)

In [8]:
rt = final_chain.invoke("오늘의 증시")
rt

'오늘 국내 증시는 하락세를 보였습니다. 코스피 지수는 다시 3200선 아래로 내려앉았으며, 이는 전일 기술적 반등 이후에도 세제개편안의 여파와 미국의 관세 영향이 본격화되면서 뉴욕증시 3대 지수가 일제히 하락한 데 따른 것으로 분석됩니다.\n\n또한, 개장 직후 투자자들의 관심을 받은 종목으로는 네이처셀이 있으며, 개장 5분 만에 높은 검색 비율(9.78%)을 기록하며 주목받았습니다. 현재가는 24,750원으로 전 거래일 대비 상승한 상태입니다.\n\n요약:\n\n- 코스피 지수: 3200선 아래로 하락\n- 주요 원인: 세제개편안 후폭풍, 미국 관세 영향, 뉴욕증시 하락\n- 인기 종목: 네이처셀 (개장 직후 높은 검색 비율 기록)\n\n투자자들은 세제 변화와 글로벌 시장 흐름에 주의할 필요가 있습니다.'